# CRSP dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('crsp.csv')

In [3]:
data.dtypes

PERMNO       int64
date        object
SHRCD        int64
EXCHCD       int64
TICKER      object
COMNAM      object
NAICS      float64
PERMCO       int64
CUSIP       object
DLAMT      float64
DLPDT       object
DLSTCD     float64
DISTCD     float64
DIVAMT     float64
FACPR      float64
FACSHR     float64
DLRETX      object
DLPRC      float64
DLRET       object
NSDINX     float64
BIDLO      float64
ASKHI      float64
PRC        float64
VOL          int64
RET         object
SHROUT       int64
CFACPR     float64
CFACSHR    float64
SPREAD     float64
vwretd     float64
dtype: object

In [4]:
data["RET"] = data["RET"].replace('C',0) # Corrupted return data set to 0
data["RET"] = data["RET"].astype('float64') # Conversion of RET: object -> float64 (string -> numeric)
data["ME"] = data["PRC"] * data["SHROUT"] / 1000
print("Min and max of ME: ({},{})".format(data["ME"].min(),(data["ME"].max())))

Min and max of ME: (0.13806000000000002,1304764.72281735)


In [5]:
def date_conversion(date):
    # Changing date format DDMMMYYYY -> YYYYMM i.e. 30JAN2015 -> 201501
    c = {'JAN' : '01', 'FEB' : '02', 'MAR' : '03', 'APR' : '04',
         'MAY' : '05', 'JUN' : '06', 'JUL' : '07', 'AUG' : '08',
         'SEP' : '09', 'OCT' : '10', 'NOV' : '11', 'DEC' : '12'}
    return date[5:] + c[date[2:5]]

v_date_conversion = np.vectorize(date_conversion)

In [6]:
print("Number of unique tickers: {}".format(data["TICKER"].nunique()))
data["date"] = v_date_conversion(data["date"])
print("Data range (date): {} to {}".format(data["date"].unique()[0], data["date"].unique()[-1]))
months = data["date"].unique()
print("Number of months: {}".format(months.size))
months

Number of unique tickers: 5058
Data range (date): 201501 to 201912
Number of months: 60


array(['201501', '201502', '201503', '201504', '201505', '201506',
       '201507', '201508', '201509', '201510', '201511', '201512',
       '201601', '201602', '201603', '201604', '201605', '201606',
       '201607', '201608', '201609', '201610', '201611', '201612',
       '201701', '201702', '201703', '201704', '201705', '201706',
       '201707', '201708', '201709', '201710', '201711', '201712',
       '201801', '201802', '201803', '201804', '201805', '201806',
       '201807', '201808', '201809', '201810', '201811', '201812',
       '201901', '201902', '201903', '201904', '201905', '201906',
       '201907', '201908', '201909', '201910', '201911', '201912'],
      dtype=object)

In [13]:
def print_ret_table(df,first=3,last=3): return df.head(first).append(df.tail(last)) 
# helper for printing return tables for debugging

# Market value breakpoints

ME is price times shares outstanding (divided by 1,000,000) at month end.

In [8]:
breakpoints = pd.read_csv('ME_Breakpoints.csv',header=None,skiprows=1).rename(columns={0 : 'date'})
breakpoints = breakpoints.loc[1069:(1069+5*12-1)].reset_index()

In [9]:
print('Max and min 100% percentile breakpoints: ({},{})'.format(breakpoints[21].min(),breakpoints[21].max()))
print_ret_table(breakpoints)

Max and min 100% percentile breakpoints: (321191.38,553773.25)


,index,date,1,2,3,4,5,6,7,8,...,12,13,14,15,16,17,18,19,20,21
0,1069,201501,1341.0,172.54,298.34,444.25,585.54,779.98,1056.65,1381.79,...,3302.10,3906.23,4893.42,6462.86,8339.49,10833.28,16621.37,25676.59,48413.76,370182.50
1,1070,201502,1338.0,184.65,321.34,475.15,634.81,849.76,1164.94,1473.23,...,3486.73,4133.06,5124.59,6883.27,8869.60,11496.35,16816.82,27426.48,50140.73,371397.84
2,1071,201503,1332.0,190.22,315.79,481.98,644.13,866.37,1198.06,1480.68,...,3519.21,4176.21,5214.60,6929.15,8785.98,11427.68,16872.71,27569.83,48549.13,356548.66
57,1126,201910,1206.0,144.79,267.96,423.71,638.88,891.88,1175.80,1533.27,...,3802.58,4896.10,5775.65,8066.26,10506.56,14291.26,21047.95,34477.98,63052.75,521148.69
58,1127,201911,1204.0,150.47,266.28,430.04,659.77,908.04,1233.44,1585.20,...,3949.01,4982.46,6112.37,8387.76,10885.17,15059.09,21955.21,35503.64,64680.46,538756.88
59,1128,201912,1189.0,165.02,304.02,452.44,745.37,990.02,1297.37,1651.30,...,4122.56,5243.03,6273.43,8492.98,11123.07,15181.22,22400.21,36860.46,68168.56,553773.25


# Factors dataset

I use data from https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html.
From section "Developed Markets Factors and Returns" file Fama/French North American 5 Factors.

In [10]:
factors = pd.read_csv('North_America_5_Factors.csv',skiprows=6)

In [11]:
factors = factors.loc[294:(294+5*12-1)].reset_index() # 201501 to 201912
factors.dtypes

index          int64
Unnamed: 0    object
Mkt-RF        object
SMB           object
HML           object
RMW           object
CMA           object
RF            object
dtype: object

In [12]:
factors = factors.drop('Unnamed: 0',1).astype('float64').multiply(1/100) # scale entries to percentages
print_ret_table(factors)

,index,Mkt-RF,SMB,HML,RMW,CMA,RF
0,2.94,-0.0347,-0.0119,-0.0389,0.0214,-0.0111,0.0000
1,2.95,0.0615,0.0057,-0.0136,-0.0105,-0.0196,0.0000
2,2.96,-0.0120,0.0250,-0.0093,0.0000,-0.0056,0.0000
57,3.51,0.0183,0.0002,-0.0147,0.0105,-0.0094,0.0015
58,3.52,0.0380,0.0021,-0.0279,-0.0167,-0.0139,0.0012
59,3.53,0.0279,0.0082,0.0143,-0.0020,0.0155,0.0014
